In [1]:
import os
import dill

import numpy as np

import TwoPUtils
import STX3KO_analyses as stx



%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
def update_sess_dict(mouse, f, KO = True):
    scan_str = "%s_%03d_%03d" % (f['scene'],f['session'],f['scan'])
    if KO:
        try:
            source_folder  =os.path.join('/media','mplitt','Backup Plus','2P_Data')
            source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
            info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
        except:
            source_folder  = os.path.join('/media','mplitt','Backup Plus2','2P_Data')
            source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
            info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
    else:
        try:
            source_folder  =os.path.join('/media','mplitt','Backup Plus3')
            source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
            info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
        except:
            source_folder  = os.path.join('/media','mplitt','Backup Plus5','2P_Data')
            source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
            info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')

    f['mouse']=mouse
    f.update({'scan_file': source_stem + '.sbx',
              'scanheader_file': source_stem + '.mat',
              'vr_filename': os.path.join("/home/mplitt/VR_scratch",f['mouse'],f['date'],"%s_%d.sqlite" %(f['scene'],f['session'])),
              'scan_number': f['scan'],
              'prompt_for_keys': False,
              'VR_only': False,
              'scanner': "NLW",
             })
    return f

In [3]:
def run_all_sessions(sess_dict,KO=True):
    failed_sessions = []
    for mouse, file_list in sess_dict.items():
        for f in file_list:
            if isinstance(f, dict):
                f = update_sess_dict(mouse,f,KO=KO)
                scan_str = "%s_%03d_%03d" % (f['scene'],f['session'],f['scan'])
                if mouse in ['4467975.5', '4467333.1']:
                    f['s2p_path']=os.path.join("/home/mplitt/2P_scratch",f['mouse'],f['date'],f['scene'],scan_str,'suite2p')
                else:
                    f['s2p_path']=os.path.join("/mnt/BigDisk/2P_scratch",f['mouse'],f['date'],f['scene'],scan_str,'suite2p')

                try:
                    sess = stx.session.YMazeSession(**f)
                    sess.load_scan_info()
                    sess.align_VR_to_2P()
                    sess.load_suite2p_data(which_ts=('F', 'Fneu'))
                    sess.get_trial_info()
                    sess.neuropil_corrected_dff()
                    sess.place_cells_calc()
                    TwoPUtils.sess.save_session(sess,'/home/mplitt/YMazeSessPkls')
                except:
                    failed_sessions.append(f)

            elif isinstance(f, tuple) or isinstance(f, list):
                if mouse in ['4467975.5', '4467333.1']:
                    s2p_path=os.path.join("/home/mplitt/2P_scratch",mouse,f[0]['date'],'combined','suite2p')
                else:
                    print(f)
                    s2p_path=os.path.join("/mnt/BigDisk/2P_scratch",mouse,f[0]['date'],'combined','suite2p')

                
                ops = np.load(os.path.join(s2p_path,"plane0","ops.npy"),allow_pickle=True).all()
                comb_frames = 0
                for _f in f:

                    _f = update_sess_dict(mouse, _f,KO=KO)
                    if mouse in ['4467975.5', '4467333.1']:
                        _f['s2p_path']=os.path.join("/home/mplitt/2P_scratch",f[0]['mouse'],f[0]['date'],'combined','suite2p')
                    else:
                        _f['s2p_path']=os.path.join("/mnt/BigDisk/2P_scratch",f[0]['mouse'],f[0]['date'],'combined','suite2p')

                    try:
                        sess = stx.session.YMazeSession(**_f)
                        sess.load_scan_info()
                        sess.align_VR_to_2P()
                        print(slice(comb_frames,comb_frames+sess.scan_info['max_idx']))
                        sess.load_suite2p_data(which_ts=('F', 'Fneu'), frames = slice(comb_frames,comb_frames+sess.scan_info['max_idx']))

                        sess.get_trial_info()
                        sess.neuropil_corrected_dff()
                        sess.place_cells_calc()
                        TwoPUtils.sess.save_session(sess,'/home/mplitt/YMazeSessPkls')

                        comb_frames += sess.scan_info['max_idx']

                    except:
                        failed_sessions.append(_f)
                print("ops max idx %d, combined sbx max idx %d" % (ops['nframes'], comb_frames))



            else:
                failed_sessions.append(['wrong type',f])

    return failed_sessions

In [4]:
def check_for_pickle(sess_dict):
    for mouse, file_list in sess_dict.items():
        for f in file_list:
            if isinstance(f,list) or isinstance(f,tuple):
                for _f in f:
                    if not os.path.exists(os.path.join("/home/mplitt/YMazeSessPkls",mouse,_f['date'],"%s_%d.pkl" % (_f['scene'],_f['session']))):
                        print('failed',_f)
            else:
                if not os.path.exists(os.path.join("/home/mplitt/YMazeSessPkls",mouse,f['date'],"%s_%d.pkl" % (f['scene'],f['session']))):
                    print('failed',_f)
                                                    
        

In [5]:
check_for_pickle(stx.ymaze_sess_deets.KO_sessions)

failed {'date': '28_09_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3, 'novel': -1}


In [28]:
check_for_pickle(stx.ymaze_sess_deets.CTRL_sessions)

In [6]:
KO_failed_sessions = run_all_sessions(stx.ymaze_sess_deets.KO_sessions)

({'date': '28_09_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3, 'novel': -1}, {'date': '28_09_2020', 'scene': 'YMaze_LNovel', 'session': 6, 'scan': 14, 'novel': -1})


/home/mplitt/repos/TwoPUtils/TwoPUtils/sess.py:95: UserWarning: skipping checking keys, remaining initialization not guaranteed to work
  warnings.warn("skipping checking keys, remaining initialization not guaranteed to work")


Fixing teleports
(10496,) (4245,)
274.5914603335018 274.49823143001515
slice(0, 4245, None)
227 3938


/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:146: RuntimeWarning: Mean of empty slice
  SI = spatial_info(np.nanmean(C_trial_mat,axis=0),occ)
/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:106: RuntimeWarning: Mean of empty slice
  P_map = P_map/np.nanmean(P_map, axis=0,keepdims=True)
/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:157: RuntimeWarning: Mean of empty slice
  _SI_perm =  spatial_info(np.nanmean(C_trial_mat,axis=0),occ)


perm 0
perm 0


...

/home/mplitt/repos/TwoPUtils/TwoPUtils/sess.py:95: UserWarning: skipping checking keys, remaining initialization not guaranteed to work
  warnings.warn("skipping checking keys, remaining initialization not guaranteed to work")


Fixing teleports
(92401,) (36648,)
2370.3731682668013 2370.2961091460334


/home/mplitt/repos/TwoPUtils/TwoPUtils/sess.py:95: UserWarning: skipping checking keys, remaining initialization not guaranteed to work
  warnings.warn("skipping checking keys, remaining initialization not guaranteed to work")


Fixing teleports
(95536,) (37853,)
2448.3401970692266 2448.234461849419


FileNotFoundError: [Errno 2] No such file or directory: '/media/mplitt/Backup Plus2/2P_Data/4467975.1/02_10_2020/YMaze_LNovel/YMaze_LNovel_001_003.mat'

In [15]:
CTRL_failed_sessions = run_all_sessions(stx.ymaze_sess_deets.CTRL_sessions,KO=False)

/home/mplitt/repos/TwoPUtils/TwoPUtils/sess.py:93: UserWarning: skipping checking keys, remaining initialization not guaranteed to work
  warnings.warn("skipping checking keys, remaining initialization not guaranteed to work")
/home/mplitt/repos/TwoPUtils/TwoPUtils/sess.py:256: UserWarning: Looking for coaligned suite2p sessions is not implemented yet
  warnings.warn("Looking for coaligned suite2p sessions is not implemented yet")


Fixing teleports
(97861,) (34310,)
2219.147296614452 2219.0763011622034
174 4674
5614 9802
10740 14950
15888 19974
20911 25580
26520 34308


/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:139: RuntimeWarning: Mean of empty slice
  SI = spatial_info(np.nanmean(C_trial_mat,axis=0),occ)
/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:150: RuntimeWarning: Mean of empty slice
  _SI_perm =  spatial_info(np.nanmean(C_trial_mat,axis=0),occ)


perm 0


/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:139: RuntimeWarning: Mean of empty slice
  SI = spatial_info(np.nanmean(C_trial_mat,axis=0),occ)


...

perm 0


/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:150: RuntimeWarning: Mean of empty slice
  _SI_perm =  spatial_info(np.nanmean(C_trial_mat,axis=0),occ)
/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:139: RuntimeWarning: Mean of empty slice
  SI = spatial_info(np.nanmean(C_trial_mat,axis=0),occ)


perm 0


/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:150: RuntimeWarning: Mean of empty slice
  _SI_perm =  spatial_info(np.nanmean(C_trial_mat,axis=0),occ)


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/BigDisk/2P_scratch/4467333.1/06_12_2020/combined/suite2p/plane0/ops.npy'

In [ ]:
print(KO_failed_sessions)

In [16]:
from numpy.random import default_rng
rng = default_rng()print(CTRL_failed_sessions)

NameError: name 'CTRL_failed_sessions' is not defined

In [5]:
def update_session(sess):
    sess.rm_timeseries('F_dFF')
    sess.neuropil_corrected_dff()
    sess.rm_pos_binned_trial_matrix('F_dff')
    sess.add_pos_binned_trial_matrix('F_dff')
    # add normalized timeseries. 
    # divide dff by 99th percentile for that cell on that session
    sess.add_timeseries(F_dff_norm=sess.timeseries['F_dff']/np.nanpercentile(sess.timeseries['F_dff'],99,axis=1,keepdims=True))
    sess.add_pos_binned_trial_matrix('F_dff_norm')
    sess.add_pos_binned_trial_matrix('spks')
    sess.add_timeseries(spks_norm = sess.timeseries['spks']/np.nanpercentile(sess.timeseries['spks'],99,axis=1,keepdims=True))
    sess.add_pos_binned_trial_matrix('spks_norm')
#     sess.place_cells_calc(impute_nans=False,bin_size=1)
    TwoPUtils.sess.save_session(sess,'/home/mplitt/YMazeSessPkls')

In [8]:
for sess_dict in [stx.ymaze_sess_deets.KO_sessions_ravel, stx.ymaze_sess_deets.CTRL_sessions_ravel]:
    for mouse, sessions in sess_dict.items():
        print(mouse)
        for i, deets in enumerate(sessions):
            print(i)
            with open(os.path.join('/home/mplitt/YMazeSessPkls',mouse,deets['date'], "%s_%d.pkl" % (deets['scene'],deets['session'])),'rb') as file:
                sess = dill.load(file)
            update_session(sess)



4467975.1
0
227 3938
1
164 4456
5396 9745
10682 15385
16326 20668
21606 29952
2
260 5104
6041 10792
11744 16594
17535 22195
23133 28008
28944 38342
3
193 4874
5811 10585
11525 16163
17099 21584
22519 27422
28362 36636
4
172 4877
5815 10445
11386 16106
17044 21907
22846 27555
28494 37841
5
192 4676
5613 9931
10868 15135
16072 20287
21227 25146
26083 34898
6
171 4733
5671 10336
11275 15933
16873 21527
22466 27346
28287 37648
7
166 4944
5879 10759
8
167 4862
5801 10613
11552 16380
17316 26724
9
186 4936
5871 10575
11514 16296
17238 22028
22966 27285
28225 36957
10
171 4871
5810 10524
11
168 5028
5969 10637
11577 16465
17406 26821
4467975.2
0
181 4249
5183 9010
9945 13818
14753 18464
19417 24005
24942 33471
1
184 4815
5905 10492
11427 16238
17246 22013
23065 27691
28631 38186
2
185 4816
5751 10444
11386 16064
17001 21970
22915 27763
28710 38179
3
164 5081
6027 10686
11628 16276
17242 22192
23134 27837
28791 38136
4
185 4967
5910 10599
11547 16265
17207 22194
23136 27948
28889 38404
5
185 4

In [7]:
mouse = '4467332.2'
for i, deets in enumerate(stx.ymaze_sess_deets.CTRL_sessions_ravel[mouse]):
    print(i)
    with open(os.path.join('/home/mplitt/YMazeSessPkls',mouse,deets['date'], "%s_%d.pkl" % (deets['scene'],deets['session'])),'rb') as file:
        sess = dill.load(file)
    update_session(sess)

0
0 4338
5278 9187
10129 12721
1
184 4092
5029 9134
10070 13530
14469 22650
2
214 4949
5884 9555
3
213 4918
5862 9924
10866 14835
15771 23024
4
180 4304
5240 9010
10048 13993
14928 18798
19734 23278
24217 31510
5
190 4274
5211 9104
10056 13792
14726 18627
19563 23947
24883 31537
6
191 4970
5915 10667
11603 15902
16841 20693
21641 25441
26486 33458
7
178 4805
5741 10456
11391 16002
16939 21486
22432 26768
27703 35783
8
169 4994
5933 10150
9
167 4869
5815 10148
11083 15135
16070 24406
10
197 4970
5905 10641
11577 16257
17196 21899
22833 27210
28144 37537
11
188 4870
5806 10641
12
174 4971
5911 10587
11524 16320
17257 26608
